<a href="https://colab.research.google.com/github/ahmet3242/Mathematical_Modellization/blob/master/Sentiment_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import pandas as pd
import numpy as np
import re #removing non string data
import io
import string #removing whole punctuations
import nltk  #stopwords
import matplotlib.pyplot as plt

from plotly import graph_objs as go
from textblob import TextBlob# Create a function to get the subjectivity
from wordcloud import WordCloud
from PIL import Image
from nltk.corpus import stopwords #nltk.download('stopwords')
from nltk import pos_tag #For lemmatize it chooses the which kind of word
from nltk.tokenize import WhitespaceTokenizer
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet

from google.colab import files
data_to_load = files.upload()

Saving vaccination_tweets.csv to vaccination_tweets.csv


In [ ]:
twits_vaccines=pd.read_csv(io.BytesIO(data_to_load['vaccination_tweets.csv']))
twits_vaccines.head()

In [4]:
twits_vaccines.dropna(subset = ["hashtags"], inplace=True)

In [5]:
twits_vaccines["text"] = twits_vaccines["text"].apply(lambda x: x.lower())  #Transforming to all characters into lower
# PreProcessing data
def cleanTxt(text):
  text = re.sub('@[A-Za-z0–9]+', '', text) #Removing @mentions
  text = re.sub('#', '', text) # Removing '#' hash tag
  text = re.sub('RT[\s]+', '', text) # Removing RT
  text = re.sub('https://\/\/\S+', '', text) # Removing hyperlink
  text = re.sub("pfizerbiontech",'', text) # removes the pfizerbiontech from pfizerbiontech, pfizerbiontech high frequency word
  text=re.sub(r'\d+', "", text) # removes the numerical datas
  return text

In [6]:
twits_vaccines["text"] = twits_vaccines["text"].apply(cleanTxt)
def remove_url(text_data): #Removes https:+URL
  return re.sub(r"http\S+", "", text_data)

twits_vaccines["text"] = twits_vaccines["text"].apply(remove_url)
twits_vaccines["text"] = twits_vaccines["text"].apply(lambda x: x.replace("\r", " "))
twits_vaccines["text"]= twits_vaccines["text"].apply(lambda x: x.replace("\n", " "))

In [7]:
nltk.download("stopwords")
stopwords.words('english')
stop_words = stopwords.words('english')
twits_vaccines["text"]= twits_vaccines["text"].apply(lambda x: " ".join(x for x in x.split() if x not in stop_words))
# if you wanna look at stop words, run this code; stopwords.words('english')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
# return the wordnet object value corresponding to the POS tag &Lemmatization&
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')
def get_wordnet_pos(pos_tag):
    if pos_tag.startswith('J'):
        return wordnet.ADJ
    elif pos_tag.startswith('V'):
        return wordnet.VERB
    elif pos_tag.startswith('N'):
        return wordnet.NOUN
    elif pos_tag.startswith('R'):
        return wordnet.ADV
    else:
        return wordnet.NOUN
    
def lemmatize_text(text):
    # tokenize text and remove puncutation
    text = [word.strip(string.punctuation) for word in text.split(" ")]
    # remove empty tokens
    text = [t for t in text if len(t) > 0]
    # pos tag text
    pos_tags = pos_tag(text)
    # lemmatize text
    text = [WordNetLemmatizer().lemmatize(t[0], get_wordnet_pos(t[1])) for t in pos_tags]
    # remove words with only one letter
    text = [t for t in text if len(t) > 1]
    # join all
    text = " ".join(text)  #Aralarda boşluk bırakarak ayırdığımız string tipi verileri birleştirir.
    return(text)

twits_vaccines["clean_data"] = twits_vaccines["text"].apply(lambda x: lemmatize_text(x))
twits_vaccines


In [9]:
from collections import Counter
cnt = Counter()
for text in twits_vaccines["clean_data"].values: 
    for word in text.split():
        cnt[word] += 1       
cnt.most_common(10)

[('vaccine', 2981),
 ('covid', 1902),
 ('dose', 940),
 ('get', 861),
 ('covidvaccine', 781),
 ('first', 678),
 ('pfizer', 589),
 ('receive', 430),
 ('today', 414),
 ('vaccination', 376)]

In [10]:
# Removing the frequent words
freq = set([w for (w, wc) in cnt.most_common(2)])
# function to remove the frequent words
def freqwords(text):
    return " ".join([word for word in str(text).split() if word not 
in freq])
# Passing the function freqwords
twits_vaccines["clean_data"] = twits_vaccines["clean_data"].apply(freqwords)
twits_vaccines["clean_data"].head(10)

0      folk say daikon paste could treat cytokine storm
2     coronavirus sputnikv astrazeneca moderna russi...
4                 explain need whereareallthesickpeople
6     bit sad claim fame success vaccination patriot...
7        many bright day best bidenharris win election…
8                           get it covidvaccine moderna
9     covidvaccine state start get covidvaccine mond...
10        death close mark million people wait first u…
11    covidvaccine corona bbcnews nytimes bbcnews be...
13                 woman healthcare provider ask safety
Name: clean_data, dtype: object

In [11]:
allWords = ' '.join([twts for twts in twits_vaccines["clean_data"]])

In [12]:
custom_mask=np.array(Image.open("vaccine.jpg")) #converting our images into np.array

# this line will take all values greater than 3 and make them 255 (white)
# if they are less than 3, they will be whatever value they are in the array
custom_mask=np.where(custom_mask > 10, 255, custom_mask)
wordcloud = WordCloud(background_color='black',width = 3000, 
                      height = 3000, 
                      max_font_size = 100, 
                      mask = custom_mask,
                      contour_width=2 ,
                      contour_color='white',colormap="Blues"
                      ).generate(allWords)

plt.figure(figsize = (12,8))
plt.imshow(wordcloud, interpolation = "bilinear")
plt.axis("off")
plt.show()

FileNotFoundError: ignored

# Yeni Bölüm

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [13]:

def getSubjectivity(text):
   return TextBlob(text).sentiment.subjectivity

# Create a function to get the polarity
def getPolarity(text):
   return  TextBlob(text).sentiment.polarity

twits_vaccines['Subjectivity'] = twits_vaccines["clean_data"].apply(getSubjectivity)
twits_vaccines['Polarity'] = twits_vaccines["clean_data"].apply(getPolarity)
# Show the new dataframe with columns 'Subjectivity' & 'Polarity'
twits_vaccines.head(20)

,id,user_name,user_location,user_description,user_created,user_followers,user_friends,user_favourites,user_verified,date,text,hashtags,source,retweets,favorites,is_retweet,clean_data,Subjectivity,Polarity
0,1340539111971516416,Rachel Roh,"La Crescenta-Montrose, CA",Aggregator of Asian American news; scanning di...,2009-04-08 17:52:46,405,1692,3247,False,2020-12-20 06:06:44,folks said daikon paste could treat cytokine s...,['PfizerBioNTech'],Twitter for Android,0,0,False,folk say daikon paste could treat cytokine storm,0.000000,0.000000
2,1337858199140118533,eli🇱🇹🇪🇺👌,Your Bed,"heil, hydra 🖐☺",2020-06-25 23:30:28,10,88,155,False,2020-12-12 20:33:45,coronavirus sputnikv astrazeneca moderna covid...,"['coronavirus', 'SputnikV', 'AstraZeneca', 'Pf...",Twitter for Android,0,0,False,coronavirus sputnikv astrazeneca moderna russi...,0.033333,0.000000
4,1337854064604966912,Citizen News Channel,NaN,Citizen News Channel bringing you an alternati...,2020-04-23 17:58:42,152,580,1473,False,2020-12-12 20:17:19,explain need vaccine whereareallthesickpeople …,"['whereareallthesickpeople', 'PfizerBioNTech']",Twitter for iPhone,0,0,False,explain need whereareallthesickpeople,0.000000,0.000000
6,1337851215875608579,Gunther Fehlinger,"Austria, Ukraine and Kosovo",End North Stream 2 now - the pipeline of corru...,2013-06-10 17:49:22,2731,5001,69344,False,2020-12-12 20:06:00,bit sad claim fame success vaccination patriot...,['vaccination'],Twitter Web App,0,4,False,bit sad claim fame success vaccination patriot...,0.500000,-0.100000
7,1337850832256176136,Dr.Krutika Kuppalli,NaN,"ID, Global Health, VHF, Pandemic Prep, Emergin...",2019-03-25 04:14:29,21924,593,7815,True,2020-12-12 20:04:29,many bright days best . bidenharris winning el...,"['BidenHarris', 'Election2020']",Twitter for iPhone,2,22,False,many bright day best bidenharris win election…,0.500000,0.750000
8,1337850023531347969,Erin Despas,NaN,Designing&selling on Teespring. Like 90s Disne...,2009-10-30 17:53:54,887,1515,9639,False,2020-12-12 20:01:16,covid vaccine; getting it? covidvaccine covid ...,"['CovidVaccine', 'covid19', 'PfizerBioNTech', ...",Twitter Web App,2,1,False,get it covidvaccine moderna,0.000000,0.000000
9,1337842295857623042,Ch.Amjad Ali,Islamabad,#ProudPakistani #LovePakArmy #PMIK @insafiansp...,2012-11-12 04:18:12,671,2368,20469,False,2020-12-12 19:30:33,covidvaccine states start getting covidvaccine...,"['CovidVaccine', 'COVID19Vaccine', 'US', 'paku...",Twitter Web App,0,0,False,covidvaccine state start get covidvaccine mond...,0.000000,0.000000
10,1337841934170255365,Tamer Yazar,Turkey-Israel,"Im Market Analyst, also Editor... working (fre...",2009-09-17 16:45:16,1302,78,339,False,2020-12-12 19:29:07,"deaths closing , mark... millions people wait ...","['PfizerBioNTech', 'Vaccine']",Twitter Web App,0,0,False,death close mark million people wait first u…,0.333333,0.250000
11,1337840331522453507,VoiceM,NaN,campaigner & optimistic realist,2020-08-31 10:38:21,2,25,20,False,2020-12-12 19:22:45,covid covidvaccine vaccine corona bbcnews nyti...,"['COVID19', 'CovidVaccine', 'vaccine', 'Corona...",Twitter Web App,0,0,False,covidvaccine corona bbcnews nytimes bbcnews be...,0.300000,1.000000
13,1337809196453081088,Dr.Krutika Kuppalli,NaN,"ID, Global Health, VHF, Pandemic Prep, Emergin...",2019-03-25 04:14:29,21924,593,7815,True,2020-12-12 17:19:02,women healthcare providers asking safety …,['PfizerBioNTech'],Twitter for iPhone,48,82,False,woman healthcare provider ask safety,0.000000,0.000000


# Yeni Bölüm

In [ ]:
# Create a function to compute negative (-1), neutral (0) and positive (+1) analysis
def getAnalysis(score):
  if score < 0:
    return "Negative"
  elif score == 0:
    return 'Neutral'
  else:
    return 'Positive'
twits_vaccines["Score"]= twits_vaccines["Polarity"].apply(getAnalysis)
# Show the dataframe
twits_vaccines.head(10)

In [ ]:
balance_counts = twits_vaccines.groupby('Score')['Score'].agg('count').values
balance_counts

array([ 585, 3208, 2340])

In [ ]:
#importing plot colors
primary_blue = "#496595"
primary_blue2 = "#85a1c1"
primary_blue3 = "#3f4d63"

fig = go.Figure()
fig.add_trace(go.Bar(
    x=['Negative Twits'],
    y=[balance_counts[0]],
    name='negative',
    text=[balance_counts[0]],
    textposition='auto',
    marker_color=primary_blue
))
fig.add_trace(go.Bar(
    x=['Neutral Twits'],
    y=[balance_counts[1]],
    name='neutral',
    text=[balance_counts[1]],
    textposition='auto',
    marker_color=primary_blue2
))
fig.add_trace(go.Bar(
    x=['Positive Twits'],
    y=[balance_counts[2]],
    name='positive',
    text=[balance_counts[2]],
    textposition='auto',
    marker_color=primary_blue3
))
fig.update_layout(
    title='<span style="font-size:64px; font-family:Calibri">DATASET DISTRIBUTION BY SENTIMENTS</span>'
)
fig.show()

In [ ]:
#x = twits_vaccines['clean_data']
#y = twits_vaccines['Score']

#print(len(x), len(y))

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidfconverter=TfidfVectorizer(max_features=1000, min_df=1,max_df=0.7)
x = tfidfconverter.fit_transform(twits_vaccines["clean_data"]).toarray()
y=twits_vaccines["Score"]
print(x.shape)
print(y)

(6133, 1000)
0        Neutral
2        Neutral
4        Neutral
6       Negative
7       Positive
          ...   
8076     Neutral
8077     Neutral
8079    Positive
8080     Neutral
8081     Neutral
Name: Score, Length: 6133, dtype: object


In [ ]:
# Split into train and test sets
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(x, y,test_size=0.2, random_state=42)
print(len(x_train), len(y_train))
print(len(x_test), len(y_test))
print(x_train)


4906 4906
1227 1227
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


In [ ]:
from sklearn.svm import SVC
from sklearn.datasets import make_blobs 
from sklearn.model_selection import RandomizedSearchCV
hyper_parameters=[{'kernel':['rbf'],'gamma':[10,1,1e-1,1e-2],'C':[1,10,100,1000]}]
Classif=RandomizedSearchCV(SVC(),hyper_parameters,n_iter=10,cv=3)
Classif.fit(x_train,y_train)
print("Hyperparameters that is gonna use:")
print(Classif.best_params_)

Hyperparameters that is gonna use:
{'kernel': 'rbf', 'gamma': 0.01, 'C': 100}


In [ ]:
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
optimalvalues=SVC(kernel=Classif.best_params_["kernel"],gamma=Classif.best_params_["gamma"],C=Classif.best_params_["C"])
optimalvalues.fit(x_train,y_train)
y_predict=optimalvalues.predict(x_test)
print(confusion_matrix(y_test,y_predict))
print(classification_report(y_test,y_predict))
print(accuracy_score(y_test,y_predict))
print(y_predict)

[[ 62  36  13]
 [  3 612  12]
 [ 14  39 436]]
              precision    recall  f1-score   support

    Negative       0.78      0.56      0.65       111
     Neutral       0.89      0.98      0.93       627
    Positive       0.95      0.89      0.92       489

    accuracy                           0.90      1227
   macro avg       0.87      0.81      0.83      1227
weighted avg       0.90      0.90      0.90      1227

0.9046454767726161
['Neutral' 'Neutral' 'Neutral' ... 'Neutral' 'Negative' 'Positive']
